In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Wed May  3 09:32:28 2017

@author: zhouyi
"""

import glob
import os
import sys

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse
import pandas as pd

import expomf

DATA_ROOT = '/Users/zhouyi/Documents/GraduatePJ/steam_data'

In [2]:
unique_uid = list()
with open(os.path.join(DATA_ROOT, 'unique_user_id.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
        
unique_gname = list()
with open(os.path.join(DATA_ROOT, 'unique_game_name.txt'), 'r') as f:
    for line in f:
        unique_gname.append(line.strip())

n_games = len(unique_gname)
n_users = len(unique_uid)

In [3]:
#Load the data and train the model
def load_data(csv_file, shape=(n_users, n_games)):
    st = pd.read_csv(csv_file)
    rows, cols = np.array(st['user_id'], dtype=np.int32), np.array(st['game_name'], dtype=np.int32)
    count = st['value']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols

train_data, rows, cols = load_data(os.path.join(DATA_ROOT, 'train.num.sub.csv'))
print 'train_data.shape:',train_data.shape
print 'train_data entries:',train_data.data.shape

vad_data, rows_vad, cols_vad = load_data(os.path.join(DATA_ROOT, 'vad.num.sub.csv'))
print 'vad__data.shape:',vad_data.shape
print 'vad_data entries:',vad_data.data.shape

test_data, rows_test, cols_test = load_data(os.path.join(DATA_ROOT, 'test.num.sub.csv'))
print 'test_data.shape:',test_data.shape
print 'test_data entries:',test_data.data.shape

print 'sparse according to vad_data:',float(vad_data.data.shape[0])/(vad_data.shape[0]*vad_data.shape[1])

train_data.shape: (1269, 1197)
train_data entries: (54310,)
vad__data.shape: (1269, 1197)
vad_data entries: (6065,)
test_data.shape: (1269, 1197)
test_data entries: (15152,)
sparse according to vad_data: 0.00399277679357


In [5]:
n_components = 50
max_iter = 20
n_jobs = 5
# grid search on init_mu = {0.1, 0.05, 0.01, 0.005, 0.001} from validation set
init_mu = 0.005
lam = 1e-5

save_dir="ExpoMF_TSP_K%d_lam%1.0E_initmu%1.0E" % (n_components, lam, init_mu)

coder = expomf.ExpoMF(n_components=n_components, max_iter=max_iter, batch_size=10, init_std=0.01, 
                      n_jobs=n_jobs, random_state=98765, save_params=True, 
                      save_dir=save_dir, early_stopping=False, verbose=True, 
                      lam_y=1., lam_theta=lam, lam_beta=lam, a=1., b=1., init_mu=init_mu)

coder.fit(train_data, vad_data=vad_data, k=20)

ITERATION #0
	Updating user factors: time=1.19
	Updating item factors: time=0.91
	Updating exposure prior: time=0.12
	Validation NDCG@k: 0.0112
ITERATION #1
	Updating user factors: time=0.81
	Updating item factors: time=0.80
	Updating exposure prior: time=0.09
	Validation NDCG@k: 0.0179
ITERATION #2
	Updating user factors: time=0.82
	Updating item factors: time=0.80
	Updating exposure prior: time=0.09
	Validation NDCG@k: 0.0208
ITERATION #3
	Updating user factors: time=1.02
	Updating item factors: time=0.71
	Updating exposure prior: time=0.09
	Validation NDCG@k: 0.0207
ITERATION #4
	Updating user factors: time=0.82
	Updating item factors: time=0.82
	Updating exposure prior: time=0.09
	Validation NDCG@k: 0.0217
ITERATION #5
	Updating user factors: time=0.60
	Updating item factors: time=0.60
	Updating exposure prior: time=0.08
	Validation NDCG@k: 0.0229
ITERATION #6
	Updating user factors: time=0.81
	Updating item factors: time=1.01
	Updating exposure prior: time=0.07
	Validation NDCG@k:

ExpoMF(batch_size=10, early_stopping=False, init_std=0.01, max_iter=20,
    n_components=50, n_jobs=5, random_state=98765,
    save_dir='ExpoMF_TSP_K50_lam1E-05_initmu5E-03', save_params=True,
    verbose=True)

In [ ]:
''' More Results are summarized here:

Validation Recall@k: 0.18848559022213601
Validation MAP@k: 0.022360898847808695

Test NDCG@k: 0.061967890168069521
Test Recall@k: 0.20135546625127884
Test MAP@k: 0.056200571106548206
'''